In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import keras

if keras.backend.backend() == "torch":
    import torch
    torch.autograd.set_grad_enabled(False)

In [ ]:
import bayesflow as bf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
simulator = bf.simulators.TwoMoonsSimulator()
samples = simulator.sample((1024,))
dataset = bf.datasets.OfflineDataset(samples, batch_size=256)

In [ ]:
approximator = bf.Approximator(
    inference_network=bf.networks.FlowMatching(),
    inference_variables=["theta"],
    inference_conditions=["x"],
)

In [ ]:
approximator.compile(optimizer="AdamW")

In [ ]:
approximator.fit(dataset, epochs=100)

In [ ]:
data = {"x": keras.ops.zeros((256, 2))}
samples = approximator.sample(256, data=data)

In [ ]:
samples = {"theta1": samples["theta"][:, 0], "theta2": samples["theta"][:, 1]}

In [ ]:
sns.scatterplot(samples, x="theta1", y="theta2")
plt.show()